## Day 47 Lecture 2 Assignment

In this assignment, we will perform K-Medoids clustering using responses to a survey about student life at a university.

In [1]:
pip install pyclustering

In [2]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist, squareform
from pyclustering.cluster.kmedoids import kmedoids
import random
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

This dataset consists of 35 binary features, each corresponding to a yes/no question that characterizes the student taking the survey.

This dataset contains a large number of features, each corresponding to a survey question. The feature name summarizes the survey question, so we will not list them all out here.

Load the dataset.

In [3]:
# answer goes here
df = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Clustering/student_life_survey.csv')

df.info()
df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2958 entries, 0 to 2957
Data columns (total 36 columns):
 #   Column                                                                                                                      Non-Null Count  Dtype
---  ------                                                                                                                      --------------  -----
 0   Q2-Participated in Societies and Interest Groups                                                                            2958 non-null   int64
 1   Q2-Participated in Clubs                                                                                                    2958 non-null   int64
 2   Q2-Participated in Halls, JCRCs and/or Residential College CSCs                                                             2958 non-null   int64
 3   Q2-Participated in University organised events                                                                              2958 non-

,Q2-Participated in Societies and Interest Groups,Q2-Participated in Clubs,"Q2-Participated in Halls, JCRCs and/or Residential College CSCs",Q2-Participated in University organised events,Q3-Interested in Arts & Culture,Q3-Interested in Science & Technology,Q3-Interested in Research and independent study,Q3-Interested in Sports,"Q3-Interested in Other competitions (eg case, debates)",Q3-Interested in Entrepreneurship,Q3-Interested in Volunteering,Q3-Interested in Others,Q4-Passionate about Animal welfare,Q4-Passionate about Arts/Culture/Heritage,Q4-Passionate about Children/Youth,Q4-Passionate about Community building,"Q4-Passionate about Diversity & Inclusion (e.g. special needs, migrant worker, interfaith and intercultural understanding)",Q4-Passionate about Environmental sustainability,Q4-Passionate about Families,Q4-Passionate about Health/Well-being (e.g mental health),Q4-Passionate about Seniors,Q4-Passionate about Poverty reduction,Q4-Passionate about Education,Q4-Passionate about None of the above,Q4-Passionate about Others,Q5-Stressed about Adjustment issues,Q5-Stressed about Academic issues,Q5-Stressed about Financial issues,Q5-Stressed about Family issues,Q5-Stressed about Friendships,Q5-Stressed about Romantic relationships,Q5-Stressed about Health related issues,Q5-Stressed about Career related issues,"Q5-Stressed about My involvement in hostel, clubs, societies, interest groups, etc.",Q5-Stressed about Others,response_id
0,0,1,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,2
2,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,3
3,1,1,1,1,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,1,0,1,1,1,1,0,1,0,4
4,1,0,1,1,0,1,1,0,0,1,1,0,0,0,1,1,0,1,0,1,0,0,1,0,0,0,1,1,0,1,0,0,0,1,0,5


For our analysis, we will focus on a specific subset of the survey that is focused on stress. These questions all begin with the string 'Q5'. Filter the columns that meet this criteria (should be 10 in total).

In [4]:
# answer goes here
stress = df.filter(like='Q5', axis=1)

stress.head()


,Q5-Stressed about Adjustment issues,Q5-Stressed about Academic issues,Q5-Stressed about Financial issues,Q5-Stressed about Family issues,Q5-Stressed about Friendships,Q5-Stressed about Romantic relationships,Q5-Stressed about Health related issues,Q5-Stressed about Career related issues,"Q5-Stressed about My involvement in hostel, clubs, societies, interest groups, etc.",Q5-Stressed about Others
0,0,1,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,1,0,0
3,1,1,0,1,1,1,1,0,1,0
4,0,1,1,0,1,0,0,0,1,0


The pyclustering implementation of kmedoids supports a variety of distance metrics, but they are primarily for numeric data. We will be using SMC/Hamming dissimilarity and precomputing the similarity matrix (an alternative would be to specify a user-defined function, which you are welcome to try in addition).

We'll assume for the next step that a pair of negative values (i.e. both responses are "no") is as informative as a pair of positive values. Compute the full distance/dissimilarity matrix for the survey data using matching/hamming distance.

In [5]:
# answer goes here
ham = pdist(stress, metric='hamming')
ham_matrix = pd.DataFrame(squareform(ham))
ham_matrix

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2918,2919,2920,2921,2922,2923,2924,2925,2926,2927,2928,2929,2930,2931,2932,2933,2934,2935,2936,2937,2938,2939,2940,2941,2942,2943,2944,2945,2946,2947,2948,2949,2950,2951,2952,2953,2954,2955,2956,2957
0,0.0,0.0,0.1,0.6,0.3,0.1,0.2,0.3,0.4,0.4,0.3,0.3,0.5,0.4,0.2,0.3,0.3,0.4,0.4,0.3,0.3,0.2,0.3,0.0,0.3,0.2,0.3,0.2,0.3,0.1,0.3,0.3,0.5,0.1,0.3,0.1,0.1,0.3,0.3,0.0,...,0.2,0.2,0.2,0.1,0.0,0.4,0.1,0.2,0.1,0.0,0.4,0.0,0.1,0.2,0.1,0.2,0.2,0.1,0.0,0.1,0.3,0.1,0.0,0.4,0.0,0.1,0.2,0.1,0.2,0.2,0.1,0.3,0.2,0.3,0.3,0.2,0.0,0.0,0.2,0.2
1,0.0,0.0,0.1,0.6,0.3,0.1,0.2,0.3,0.4,0.4,0.3,0.3,0.5,0.4,0.2,0.3,0.3,0.4,0.4,0.3,0.3,0.2,0.3,0.0,0.3,0.2,0.3,0.2,0.3,0.1,0.3,0.3,0.5,0.1,0.3,0.1,0.1,0.3,0.3,0.0,...,0.2,0.2,0.2,0.1,0.0,0.4,0.1,0.2,0.1,0.0,0.4,0.0,0.1,0.2,0.1,0.2,0.2,0.1,0.0,0.1,0.3,0.1,0.0,0.4,0.0,0.1,0.2,0.1,0.2,0.2,0.1,0.3,0.2,0.3,0.3,0.2,0.0,0.0,0.2,0.2
2,0.1,0.1,0.0,0.7,0.4,0.2,0.1,0.2,0.5,0.3,0.2,0.2,0.4,0.5,0.3,0.2,0.2,0.3,0.3,0.4,0.2,0.3,0.2,0.1,0.2,0.3,0.2,0.1,0.4,0.0,0.2,0.2,0.4,0.2,0.4,0.0,0.2,0.2,0.4,0.1,...,0.3,0.1,0.1,0.0,0.1,0.3,0.0,0.1,0.2,0.1,0.3,0.1,0.2,0.3,0.2,0.1,0.1,0.0,0.1,0.0,0.2,0.2,0.1,0.3,0.1,0.2,0.3,0.0,0.3,0.1,0.2,0.4,0.3,0.4,0.4,0.3,0.1,0.1,0.1,0.1
3,0.6,0.6,0.7,0.0,0.5,0.5,0.8,0.7,0.2,0.6,0.7,0.5,0.5,0.4,0.6,0.7,0.7,0.6,0.6,0.7,0.7,0.6,0.7,0.6,0.5,0.6,0.7,0.8,0.5,0.7,0.7,0.7,0.7,0.5,0.5,0.7,0.7,0.7,0.7,0.6,...,0.6,0.6,0.8,0.7,0.6,0.4,0.7,0.6,0.7,0.6,0.4,0.6,0.5,0.6,0.5,0.8,0.6,0.7,0.6,0.7,0.5,0.5,0.6,0.6,0.6,0.7,0.6,0.7,0.4,0.6,0.7,0.5,0.4,0.5,0.3,0.6,0.6,0.6,0.6,0.8
4,0.3,0.3,0.4,0.5,0.0,0.2,0.3,0.6,0.3,0.3,0.4,0.4,0.4,0.5,0.5,0.6,0.4,0.5,0.3,0.4,0.2,0.3,0.4,0.3,0.4,0.3,0.2,0.3,0.4,0.4,0.4,0.2,0.4,0.4,0.4,0.4,0.2,0.4,0.4,0.3,...,0.3,0.3,0.3,0.4,0.3,0.3,0.4,0.5,0.2,0.3,0.3,0.3,0.2,0.3,0.4,0.5,0.5,0.4,0.3,0.4,0.4,0.4,0.3,0.5,0.3,0.2,0.3,0.4,0.3,0.5,0.2,0.2,0.3,0.4,0.2,0.3,0.3,0.3,0.5,0.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2953,0.2,0.2,0.3,0.6,0.3,0.3,0.2,0.5,0.4,0.4,0.3,0.5,0.3,0.4,0.4,0.5,0.1,0.6,0.2,0.3,0.3,0.2,0.3,0.2,0.5,0.0,0.3,0.2,0.1,0.3,0.3,0.3,0.5,0.3,0.5,0.3,0.1,0.3,0.3,0.2,...,0.2,0.4,0.2,0.3,0.2,0.6,0.3,0.4,0.1,0.2,0.6,0.2,0.3,0.4,0.3,0.4,0.2,0.3,0.2,0.3,0.5,0.1,0.2,0.4,0.2,0.1,0.2,0.3,0.4,0.4,0.1,0.1,0.4,0.3,0.5,0.0,0.2,0.2,0.4,0.2
2954,0.0,0.0,0.1,0.6,0.3,0.1,0.2,0.3,0.4,0.4,0.3,0.3,0.5,0.4,0.2,0.3,0.3,0.4,0.4,0.3,0.3,0.2,0.3,0.0,0.3,0.2,0.3,0.2,0.3,0.1,0.3,0.3,0.5,0.1,0.3,0.1,0.1,0.3,0.3,0.0,...,0.2,0.2,0.2,0.1,0.0,0.4,0.1,0.2,0.1,0.0,0.4,0.0,0.1,0.2,0.1,0.2,0.2,0.1,0.0,0.1,0.3,0.1,0.0,0.4,0.0,0.1,0.2,0.1,0.2,0.2,0.1,0.3,0.2,0.3,0.3,0.2,0.0,0.0,0.2,0.2
2955,0.0,0.0,0.1,0.6,0.3,0.1,0.2,0.3,0.4,0.4,0.3,0.3,0.5,0.4,0.2,0.3,0.3,0.4,0.4,0.3,0.3,0.2,0.3,0.0,0.3,0.2,0.3,0.2,0.3,0.1,0.3,0.3,0.5,0.1,0.3,0.1,0.1,0.3,0.3,0.0,...,0.2,0.2,0.2,0.1,0.0,0.4,0.1,0.2,0.1,0.0,0.4,0.0,0.1,0.2,0.1,0.2,0.2,0.1,0.0,0.1,0.3,0.1,0.0,0.4,0.0,0.1,0.2,0.1,0.2,0.2,0.1,0.3,0.2,0.3,0.3,0.2,0.0,0.0,0.2,0.2
2956,0.2,0.2,0.1,0.6,0.5,0.3,0.2,0.3,0.6,0.4,0.3,0.3,0.5,0.4,0.4,0.3,0.3,0.4,0.4,0.3,0.3,0.2,0.3,0.2,0.3,0.4,0.3,0.2,0.5,0.1,0.1,0.3,0.5,0.3,0.5,0.1,0.3,0.3,0.5,0.2,...,0.4,0.2,0.2,0.1,0.2,0.4,0.1,0.0,0.3,0.2,0.4,0.2,0.3,0.4,0.1,0.2,0.2,0.1,0.2,0.1,0.3,0.3,0.2,0.4,0.2,0.3,0.4,0.1,0.2,0.0,0.3,0.5,0.4,0.5,0.5,0.4,0.2,0.2,0.0,0.2


Using the dissimilarity matrix, perform kmedoids clustering using k=2. Set the initial medoids randomly. Note that pyclustering expects the distance matrix to be a numpy array; a pandas dataframe may cause errors. 

Which survey responses are chosen as the cluster representatives? Print out the details of these responses.

In [6]:
# answer goes here
num_clusters = 2

initial_medoids = stress.sample(num_clusters).index

kmedoids_instance = kmedoids(ham_matrix, initial_medoids, data_type='distance_matrix')

In [7]:
kmedoids_instance.process()

In [8]:
clusters = kmedoids_instance.get_clusters()
medoids = kmedoids_instance.get_medoids()

In [9]:
for i in range(len(clusters)):
  stress.loc[clusters[i],'cluster'] = i

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [10]:
stress.cluster.value_counts()  

0.0    2540
1.0     418
Name: cluster, dtype: int64

In [11]:
stress.loc[medoids].style.background_gradient()

,Q5-Stressed about Adjustment issues,Q5-Stressed about Academic issues,Q5-Stressed about Financial issues,Q5-Stressed about Family issues,Q5-Stressed about Friendships,Q5-Stressed about Romantic relationships,Q5-Stressed about Health related issues,Q5-Stressed about Career related issues,"Q5-Stressed about My involvement in hostel, clubs, societies, interest groups, etc.",Q5-Stressed about Others,cluster
0,0,1,0,0,0,0,0,0,0,0,0.000000
103,0,1,0,0,0,0,0,1,1,0,1.000000


If you run the previous cell a few times, you'll probably notice that the medoids are very sensitive to initialization. A common approach to produce well-separated clusters is to choose initial medoids that are far apart. Re-run the previous process, except with a random pair of medoids that have a dissimilarity of 0.8 or higher. Are the results more stable now? How would you describe the typical clusters you see?

In [12]:
ham_matrix['id'] = ham_matrix.index
sim_melted = pd.melt(ham_matrix, id_vars=['id'], value_vars=ham_matrix.columns[ham_matrix.columns != 'id'])
sim_melted.columns = ['response_id_1', 'response_id_2', 'dissimilarity']
sim_melted

,response_id_1,response_id_2,dissimilarity
0,0,0,0.0
1,1,0,0.0
2,2,0,0.1
3,3,0,0.6
4,4,0,0.3
...,...,...,...
8749759,2953,2957,0.2
8749760,2954,2957,0.2
8749761,2955,2957,0.2
8749762,2956,2957,0.2


In [13]:
sim = sim_melted.loc[lambda x: x["dissimilarity"] >= 0.8]
sim

,response_id_1,response_id_2,dissimilarity
245,245,0,0.8
263,263,0,0.8
496,496,0,0.8
752,752,0,0.8
812,812,0,0.8
...,...,...,...
8743789,2899,2955,0.8
8744952,1104,2956,0.8
8745581,1733,2956,0.8
8746809,3,2957,0.8


In [14]:
sample = sim.sample(1)

In [15]:
sample.response_id_1.values[0], sample.response_id_2.values[0]

(180, 1282)

In [16]:
initial_medoids = [sample.response_id_1.values[0], sample.response_id_2.values[0]]

kmedoids_instance = kmedoids(ham_matrix, initial_medoids, data_type='distance_matrix')

In [17]:
kmedoids_instance.process()

In [18]:
clusters = kmedoids_instance.get_clusters()
medoids = kmedoids_instance.get_medoids()

In [20]:
stress.cluster.value_counts()

0.0    2540
1.0     418
Name: cluster, dtype: int64

In [19]:
stress.loc[medoids].style.background_gradient()

,Q5-Stressed about Adjustment issues,Q5-Stressed about Academic issues,Q5-Stressed about Financial issues,Q5-Stressed about Family issues,Q5-Stressed about Friendships,Q5-Stressed about Romantic relationships,Q5-Stressed about Health related issues,Q5-Stressed about Career related issues,"Q5-Stressed about My involvement in hostel, clubs, societies, interest groups, etc.",Q5-Stressed about Others,cluster
180,0,1,0,0,0,0,0,0,0,0,0.000000
1282,1,1,1,1,1,1,1,1,1,0,1.000000


After re-running the models several times, you do see more segmentation between clusters when we set a more dissimilar initial medoid. We saw anywhere from 2-6 more categories difference between clusters when we started with more disimilair medoids. It appears that initial medoids can have a drastic impact on the model's clusters.  